# 讀取資料
情書大全: https://www.dgreetings.com/love_romance_cards/letters.html

In [1]:
import pandas as pd
Corpus = pd.read_excel("Love_letters.xlsx")

In [2]:
Corpus.head()

,ID,Text
0,1,"Dear Darling,\nI have always wanted to write y..."
1,2,"Darling,\nI want you to know that there's no o..."
2,3,"Dear,\nMy life is beautiful because of you my ..."
3,4,"Dear,\nIn life there are certain things that w..."
4,5,"To my forever person,\nI think we make the per..."


建立一個大的文本集

In [3]:
All_text = ""
for text in Corpus["Text"]:
    All_text += text

將字母轉為小寫，減少之後字元向量的維度

In [4]:
All_text = All_text.lower()

In [5]:
print(f"文本集前190字元:\n{All_text[:190]} ")

文本集前190字元:
dear darling,
i have always wanted to write you a love letter, though i don't know how to begin.
when i am with you, i am lost of words, my heart yearns for you.
you are the love of my life. 


In [6]:
print(f"文本集共{len(All_text)}個字元") 

文本集共16285個字元


# 準備訓練集
把文本集切割成許多片段，並由下一個字元當作標籤

In [7]:
segment = [] # 文本片段
next_char = [] # 該片段的下一個字元
segment_len = 50
last_segment_start_ith = len(All_text)-segment_len # 最後一個片段的
for i in range(0,last_segment_start_ith):
    segment.append(All_text[i:i+segment_len])
    next_char.append(All_text[i+segment_len])

由於我們共有16285個字元，每50個字元切割一片段，每個切割後只右移1個字元，所以理論上有16285-50 = 16235個片段

In [8]:
print("(樣本數)片段數:",len(segment),"標籤數:",len(next_char))

(樣本數)片段數: 16235 標籤數: 16235


In [9]:
import numpy as np
i = np.random.randint(len(segment))
print(f"第{i}筆訓練資料:\n{segment[i]}\n") 
print(f"第{i}筆標籤:\n{next_char[i]}") 

第10579筆訓練資料:
er.

love youmy dearest
i wanted to write you a lo

第10579筆標籤:
v


# 字元向量
把a,b,c,...,z以及一些標點符號做one-hot encoding

In [10]:
all_char = []
for i in range(len(All_text)):
    all_char.append(All_text[i])
all_unique_char = np.unique(all_char)

取得所有獨立的字元

In [11]:
print(f"獨立字元有{len(all_unique_char)}個\n{all_unique_char}")

獨立字元有39個
['\n' ' ' '!' "'" ',' '-' '.' ';' '?' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i'
 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '–'
 '—' '’' '…']


sklearn的OneHotEncoder()只能吃2d array

In [12]:
all_unique_char = np.array(all_unique_char)
all_unique_char = all_unique_char.reshape((len(all_unique_char),1)) #將39個字元轉為2d array

segment = np.array(segment)
segment = segment.reshape((len(segment),1)) #將39個字元轉為2d array

next_char = np.array(next_char)
next_char = next_char.reshape((len(next_char),1)) #將39個字元轉為2d array

In [13]:
all_unique_char[:]

array([['\n'],
       [' '],
       ['!'],
       ["'"],
       [','],
       ['-'],
       ['.'],
       [';'],
       ['?'],
       ['a'],
       ['b'],
       ['c'],
       ['d'],
       ['e'],
       ['f'],
       ['g'],
       ['h'],
       ['i'],
       ['j'],
       ['k'],
       ['l'],
       ['m'],
       ['n'],
       ['o'],
       ['p'],
       ['q'],
       ['r'],
       ['s'],
       ['t'],
       ['u'],
       ['v'],
       ['w'],
       ['x'],
       ['y'],
       ['z'],
       ['–'],
       ['—'],
       ['’'],
       ['…']], dtype='<U1')

In [14]:
next_char

array([['l'],
       ['o'],
       ['v'],
       ...,
       ['v'],
       ['e'],
       ['.']], dtype='<U1')

In [15]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe.fit(all_unique_char)
y = ohe.transform(next_char).toarray() # label

每1個字元(或稱token)現在由1個39維向量表示，所以y變成16235 x 39 維的矩陣

In [16]:
np.array(y)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

比較麻煩的是如何將segment變成 16235x50x39 的矩陣

In [17]:
# 我希望1個片段變成 50x39維的東西
text = segment[16234]
text

array(['u.\nlove you forever and always.\n\nwith lots of love'],
      dtype='<U50')

In [18]:
X = []
for i in range(len(segment)):
    text = segment[i]
    all_char_in_text = []
    for j in range(segment_len):
        all_char_in_text.append(text[0][j])
    all_char_in_text = np.array(all_char_in_text)
    all_char_in_text = all_char_in_text.reshape((-1,1))
    all_char_in_text = ohe.transform(all_char_in_text).toarray()
    #print(all_char_in_text.shape)
    X.append(all_char_in_text)

In [19]:
X = np.array(X)

趕緊看看上面的魔法成功了沒

In [20]:
# X.reshape((len(segment),segment_len,len(all_unique_char)))
X.shape

(16235, 50, 39)

In [21]:
X[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

# 建立Attention模型

In [22]:
segment_len = 50 #1個片段有50個字元
vocab = 39 #有39個字元

In [43]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten
from keras_self_attention import SeqSelfAttention,SeqWeightedAttention #前者輸出seq 後者輸出num
import keras
model = Sequential()
model.add(LSTM(128,return_sequences= True,input_shape= (segment_len, vocab)))
model.add(LSTM(128,return_sequences= True))
model.add(SeqSelfAttention(attention_activation = 'sigmoid'))
model.add(SeqWeightedAttention())
model.add(Dense(units = vocab*3, activation= "relu"))
model.add(Dense(units = vocab, activation= "softmax"))
model.compile(loss= "categorical_crossentropy", optimizer= keras.optimizers.Adadelta(), metrics=['accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_23 (LSTM)               (None, 50, 128)           86016     
_________________________________________________________________
lstm_24 (LSTM)               (None, 50, 128)           131584    
_________________________________________________________________
seq_self_attention_12 (SeqSe (None, 50, 128)           8257      
_________________________________________________________________
seq_weighted_attention_12 (S (None, 128)               129       
_________________________________________________________________
dense_22 (Dense)             (None, 117)               15093     
_________________________________________________________________
dense_23 (Dense)             (None, 39)                4602      
Total params: 245,681
Trainable params: 245,681
Non-trainable params: 0
_______________________________________________

In [44]:
model.fit(X, y, batch_size= 32, epochs= 128)

Epoch 1/128
16235/16235 [==============================] - 46s 3ms/step - loss: 2.9994 - accuracy: 0.1829
Epoch 2/128
16235/16235 [==============================] - 46s 3ms/step - loss: 2.9665 - accuracy: 0.1854 1s - loss: - ETA: 0s - loss: 2.9658 
Epoch 3/128
16235/16235 [==============================] - 46s 3ms/step - loss: 2.9639 - accuracy: 0.1854 1s - loss: - ETA: 0s - loss: 2.9645 - accuracy: 0.
Epoch 4/128
16235/16235 [==============================] - 46s 3ms/step - loss: 2.9628 - accuracy: 0.1854
Epoch 5/128
16235/16235 [==============================] - 48s 3ms/step - loss: 2.9613 - accuracy: 0.1854
Epoch 6/128
16235/16235 [==============================] - 48s 3ms/step - loss: 2.9616 - accuracy: 0.1854
Epoch 7/128
 2816/16235 [====>.........................] - ETA: 42s - loss: 2.9738 - accuracy: 0.1786

KeyboardInterrupt: 

In [ ]:
# 訓練完先保存下來! 這很重要! 
#model.save("Attention_love_letter_generator.h5")

# 預測下個字元

In [ ]:
segment[0]

In [ ]:
X_test = X[0].reshape((1,50,39))

In [ ]:
X[0]

In [ ]:
X_test

In [ ]:
pred = model.predict(X,verbose=0)[0] #預測第1筆樣本下一個字元

這些數字是機率，all_unique_char中的字元所對應的機率，機率高的字元會被挑選做預測的下一個字

In [ ]:
pred

## greedy selection + multinormial distribution
將機率縮放(開根號)，以至於常態分配比較符合我們的期望

In [ ]:
pred = pred**(0.5)
pred = pred / (np.sum(pred) + np.min(pred))

In [ ]:
pred

基本上，會挑選機率最大的字元，但這樣會減少隨機性。
因此我們會將機率擬合成多元常態分配，從該分配中抽取字元。

In [ ]:
next_onehot = np.random.multinomial(1,pred,1)
next_index = np.argmax(next_onehot)
print(next_index)
next_char_pred = all_unique_char[next_index]

In [ ]:
print("這是第一個片段:", segment[0])
print("預測下一個字元為:",next_char_pred)
print("實際下一個字元為:",next_char[0])

寫成一個function吧! 接下來可是要狂用他呢!

In [ ]:
def get_2d_input(text):
    all_char_in_text = []
    for j in range(segment_len):
        all_char_in_text.append(text[0][j])
    all_char_in_text = np.array(all_char_in_text)
    all_char_in_text = all_char_in_text.reshape((-1,1))
    all_char_in_text = ohe.transform(all_char_in_text).toarray()
    return all_char_in_text

def generate_next_char(text_input):
    """
    text_input (str) : 50 char
    """
    x_input = get_2d_input(np.array([text_input]))
    x_input = x_input.reshape((1,50,39)) # 1個片段，50個字元，39維字元向量
    #print(x_input)
    pred = model.predict(x_input,verbose=0)[0] #預測第1筆樣本下一個字元
    pred = pred**(0.5)
    pred = pred / (np.sum(pred)+ 0.000001)
    #print(pred)
    next_onehot = np.random.multinomial(1,pred,1)
    next_index = np.argmax(next_onehot)
    next_char_pred = all_unique_char[next_index]
    return next_char_pred

In [ ]:
text_input = 't like this ,before in my life. my love for you ha'
a = generate_next_char(text_input)
print("下一個字元: ",a)

# 寫情書囉!

In [ ]:
def generate_love_letter(text_input,letter_length):
    """
    text_input (str) : 50 char
    letter_length (int > 50): max number of chars in this letter(original 50 chars are included)
    output(str):
    """
    #print("初始片段\n",text_input)
    all_text_output = text_input
    while(len(all_text_output) < letter_length):
        next_char_pred = generate_next_char(text_input)
        text_input = text_input[1:] + next_char_pred[0]
        
        all_text_output = all_text_output + next_char_pred[0]
    print("生成文章\n",all_text_output)
    return all_text_output

In [ ]:
begining_text = "Dear Darling, I have always wanted to write you a love letter, though I don't know how to begin."
begining_text = begining_text.lower()
my_love_letter = generate_love_letter(begining_text[:50],200)

# 英文不好？有請谷哥翻譯

In [ ]:
import googletrans
# Initial
translator = googletrans.Translator()

# Basic Translate
results = translator.translate(my_love_letter)
print(results)

In [ ]:
#googletrans.LANGCODES

In [ ]:
print('繁體中文:\n', translator.translate(my_love_letter, dest='zh-tw').text)

# 自由創作區 7/28以後直接執行以下內容

In [ ]:
from tensorflow.keras.models import load_model
#model = load_model('Attention_love_letter_generator.h5')

In [ ]:
def Final_present():
    begining_text = input("請輸入英文句子(至少50個字，不能有數字):\n")
    begining_text = begining_text.lower()
    print("-------------------------------------------------------")
    my_love_letter = generate_love_letter(begining_text[:50],1000)
    print("-------------------------------------------------------")
    print('繁體中文:\n', translator.translate(my_love_letter, dest='zh-tw').text)

In [ ]:
Final_present()
# May your love soar on the wings of a dove in flight. My love